# Test for a random walk (CE currencies)

This notebook shows a random walk tests on the example of the EURCZK, EURPLN and EURHUF exchange rate. This test is performed first intuitively using linear regression and then numerically using the ADF test. 

In [1]:
from datetime import datetime
from datetime import date, timedelta
import pandas as pd
import yfinance as yf
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [2]:
#The analysis will be based on historical data of last two years (365*2 days)

start_date = (date.today()-timedelta(days=(365*2))).isoformat() 

## EURCZK 

In [3]:
#This part is downloading from yfinance

yfinance_ticker = 'EURCZK=X'

eurczk=yf.download(yfinance_ticker, start=start_date)

[*********************100%***********************]  1 of 1 completed


This section intuitively tests a random walk using linear regression. The tested equation is: 

$${EURCZK}_{(t)}-{EURCZK}_{(t-1)} = {const}+{coef}*{EURCZK}_{(t-1)}+{noise}$$

If the time series meets the conditions of a random walk, then the coef value should be close to zero. 

In [4]:
#The tested equation is EURCZK[t]-EURCZK[t-1]=const+coef(EURCZK[t-1])+noise 


#We need the EURCZK[t-1]

eurczk_shift=eurczk['Close'].shift(-1)
eurczk_shift=eurczk_shift[1:]
eurczk_shift

#Lets delete the first column, as it is NaN for the shifted EURCZK[t-1]

eurczk=eurczk['Close'][1:]

#Lets do the diff (the first part of the above equation)

eurczk_diff=eurczk-eurczk_shift


#Dataframe

df = pd.DataFrame({
        'eurczk_diff':eurczk_diff,
        'eurczk_shift':eurczk_shift
        
        })

df = df.dropna(axis=0)


#Linear regression

df["const"]=1

reg1 = sm.OLS(endog=df['eurczk_diff'], exog=df[['const', 'eurczk_shift']], \
    missing='drop')
type(reg1)


results = reg1.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            eurczk_diff   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.069
Date:                Sun, 31 Oct 2021   Prob (F-statistic):             0.0804
Time:                        22:33:59   Log-Likelihood:                 409.25
No. Observations:                 520   AIC:                            -814.5
Df Residuals:                     518   BIC:                            -806.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.3163      0.181      1.749   

Jak je vidět z tabulky, coef je -0.137. Tento výsledek indukuje náhodnou procházku, protože je blízko nule. Další část tento test provádí výpočtem. 

In [5]:
results = adfuller(eurczk)

result=""

if results[1]>0.05:
  result="is a random walk"
else:
  result="is not a random walk"


print("")

print(f"p-value: {results[1]}")
print(f"The time series {yfinance_ticker} {result}, because the pvalue is higher than 0.05")
#If p value is higher then 0.05, then TS is RW


p-value: 0.10733472040521091
The time series EURCZK=X is a random walk, because the pvalue is higher than 0.05


Because the pvalue is greater than 0.05, we confirm our previous assumption that the process meets the conditions of a random walk. 

## Other CE currencies 

This section tests a random walk for the other two CE currencies, EURPLN and EURHUF. 

In [6]:
yfinance_tickers = ['EURPLN=X','EURHUF=X']

for ticker in yfinance_tickers:

    data=yf.download(ticker, start=start_date)
    
    data=data["Close"]

    results = adfuller(data)

    result=""

    if results[1]>0.05:
      result="is a random walk"
    else:
      result="is not a random walk"


    print("")

    print(f"p-value: {round(results[1],2)}")
    print(f"The time series {ticker} {result}, because the pvalue is higher than 0.05")

    print("")

[*********************100%***********************]  1 of 1 completed

p-value: 0.24
The time series EURPLN=X is a random walk, because the pvalue is higher than 0.05

[*********************100%***********************]  1 of 1 completed

p-value: 0.12
The time series EURHUF=X is a random walk, because the pvalue is higher than 0.05



This test proved that the development of all Central European currencies in the last two years is a process of random walk. 

This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services. Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company. In preparing the information contained herein, author has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, freely availible at the time of publication. Author makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.